In [1]:
from simple.binance import getHistMonth
from simple.chart import chartFigure, updateFigure
from simple.pretty import pmap

import pandas as pd
from statsmodels.tsa.stattools import adfuller
from ipyslickgrid import show_grid
from ipywidgets import VBox

In [2]:
frame = '1h'

tickers = [
    '1000SHIBUSDT', '1000XECUSDT', '1INCHUSDT', 'AAVEUSDT', 'ADABUSD', 'ADAUSDT', 'ALGOUSDT', 'ALICEUSDT', 
    'ALPHAUSDT', 'ANKRUSDT', 'ANTUSDT', 'API3USDT', 'ARPAUSDT', 'ARUSDT', 'ATAUSDT', 'ATOMUSDT', 'AUDIOUSDT', 
    'AVAXUSDT', 'AXSUSDT', 'BAKEUSDT', 'BALUSDT', 'BANDUSDT', 'BATUSDT', 'BCHUSDT', 'BELUSDT', 'BLZUSDT',
    'BNBBUSD', 'BNBUSDT', 'BTCBUSD', 'BTCDOMUSDT', 'BTCUSDT', 'C98USDT', 'CELOUSDT', 'CELRUSDT',
    'CHRUSDT', 'CHZUSDT', 'COMPUSDT', 'COTIUSDT', 'CRVUSDT', 'CTKUSDT', 'CTSIUSDT', 'DASHUSDT', 'DEFIUSDT',
    'DENTUSDT', 'DGBUSDT', 'DOGEBUSD', 'DOGEUSDT', 'DOTUSDT', 'DUSKUSDT', 'DYDXUSDT', 'EGLDUSDT', 'ENJUSDT',
    'ENSUSDT', 'EOSUSDT', 'ETCUSDT', 'ETHBUSD', 'ETHUSDT', 'FILUSDT', 'FLMUSDT', 'FLOWUSDT', 'FTMUSDT', 'GALAUSDT', 
    'GRTUSDT', 'GTCUSDT', 'HBARUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IMXUSDT', 'IOSTUSDT', 'IOTAUSDT', 
    'IOTXUSDT', 'KAVAUSDT', 'KLAYUSDT', 'KNCUSDT', 'KSMUSDT', 'LINAUSDT', 'LINKUSDT', 'LITUSDT', 'LPTUSDT',
    'LRCUSDT', 'LTCUSDT', 'MANAUSDT', 'MASKUSDT', 'MATICUSDT', 'MKRUSDT', 'MTLUSDT', 'NEARUSDT', 'NEOUSDT',
    'NKNUSDT', 'OCEANUSDT', 'OGNUSDT', 'OMGUSDT', 'ONEUSDT', 'ONTUSDT', 'PEOPLEUSDT', 'QTUMUSDT', 'REEFUSDT',
    'RENUSDT', 'RLCUSDT', 'ROSEUSDT', 'RSRUSDT', 'RUNEUSDT', 'RVNUSDT', 'SANDUSDT', 'SFPUSDT', 'SKLUSDT',
    'SNXUSDT', 'SOLBUSD', 'SOLUSDT', 'STMXUSDT', 'STORJUSDT', 'SUSHIUSDT', 'SXPUSDT', 'THETAUSDT', 'TLMUSDT',
    'TOMOUSDT', 'TRBUSDT', 'TRXUSDT', 'UNFIUSDT', 'UNIUSDT', 'VETUSDT', 'WAVESUSDT', 'XEMUSDT', 'XLMUSDT', 
    'XMRUSDT', 'XRPBUSD', 'XRPUSDT', 'XTZUSDT', 'YFIUSDT', 'ZECUSDT', 'ZENUSDT', 'ZILUSDT', 'ZRXUSDT'
]

In [5]:
def getClose(ticker: str) -> pd.Series:
    return getHistMonth('2021-01', '2024-07', ticker, frame, close_only=True, spot=False)

In [6]:
# parallel load data from binance archive
Close = pd.concat(pmap(getClose, tickers, n_jobs=32), axis=1)
Close.head()

  0%|          | 0/134 [00:00<?, ?it/s]

,1000SHIBUSDT,1000XECUSDT,1INCHUSDT,AAVEUSDT,ADABUSD,ADAUSDT,ALGOUSDT,ALICEUSDT,ALPHAUSDT,ANKRUSDT,...,XLMUSDT,XMRUSDT,XRPBUSD,XRPUSDT,XTZUSDT,YFIUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
DateTime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,1.3246,88.285,NaN,0.18080,0.3460,NaN,0.18775,NaN,...,0.12910,156.99,NaN,0.2217,2.007,22870.5,64.74,11.818,0.08635,0.3575
2021-01-01 01:00:00,NaN,NaN,1.2998,87.653,NaN,0.18389,0.3450,NaN,0.19167,NaN,...,0.13326,159.95,NaN,0.2241,2.061,22954.7,65.64,11.787,0.08550,0.3612
2021-01-01 02:00:00,NaN,NaN,1.2799,88.010,NaN,0.18400,0.3548,NaN,0.19274,NaN,...,0.13295,161.35,NaN,0.2239,2.085,23185.7,65.75,11.903,0.08634,0.3626
2021-01-01 03:00:00,NaN,NaN,1.2612,88.531,NaN,0.18326,0.3593,NaN,0.18934,NaN,...,0.13114,159.75,NaN,0.2274,2.074,23234.1,65.46,12.332,0.08775,0.3611
2021-01-01 04:00:00,NaN,NaN,1.2424,87.560,NaN,0.18201,0.3531,NaN,0.18940,NaN,...,0.13363,159.79,NaN,0.2393,2.038,22878.1,65.40,12.436,0.08597,0.3595


ADAUSDT https://data.binance.vision/data/futures/um/monthly/klines/ADAUSDT/1h/ADAUSDT-1h-2022-03.zip Unknown datetime string format, unable to parse: open_time, at position 0
ADAUSDT https://data.binance.vision/data/futures/um/monthly/klines/ADAUSDT/1h/ADAUSDT-1h-2022-01.zip Unknown datetime string format, unable to parse: open_time, at position 0
ADAUSDT https://data.binance.vision/data/futures/um/monthly/klines/ADAUSDT/1h/ADAUSDT-1h-2022-02.zip Unknown datetime string format, unable to parse: open_time, at position 0
FLMUSDT https://data.binance.vision/data/futures/um/monthly/klines/FLMUSDT/1h/FLMUSDT-1h-2022-02.zip Unknown datetime string format, unable to parse: open_time, at position 0
FLMUSDT https://data.binance.vision/data/futures/um/monthly/klines/FLMUSDT/1h/FLMUSDT-1h-2022-01.zip Unknown datetime string format, unable to parse: open_time, at position 0
OMGUSDT https://data.binance.vision/data/futures/um/monthly/klines/OMGUSDT/1h/OMGUSDT-1h-2022-02.zip Unknown datetime string 

In [5]:
# parallel run of Augmented Dickey-Fuller test
ADF = pd.DataFrame(pmap(adfuller, Close.dropna().values.T),
                   index=Close.columns,
                   columns=['ADF', 'P-value', 'Used Lag', 'Observations', 'Critical Values', 'Best IC'])
ADF

  0%|          | 0/134 [00:00<?, ?it/s]

,ADF,P-value,Used Lag,Observations,Critical Values,Best IC
1000SHIBUSDT,-0.397774,0.910470,0,35,"{'1%': -3.6327426647230316, '5%': -2.948510204...",-436.733872
1000XECUSDT,-1.124412,0.705239,1,34,"{'1%': -3.639224104416853, '5%': -2.9512301791...",-381.620568
1INCHUSDT,-1.766150,0.397392,2,33,"{'1%': -3.6461350877925254, '5%': -2.954126991...",-203.666842
AAVEUSDT,-0.656802,0.857624,0,35,"{'1%': -3.6327426647230316, '5%': -2.948510204...",33.690114
ADABUSD,-0.353771,0.917536,0,35,"{'1%': -3.6327426647230316, '5%': -2.948510204...",-226.491261
...,...,...,...,...,...,...
YFIUSDT,-4.090040,0.001006,3,32,"{'1%': -3.653519805908203, '5%': -2.9572185644...",255.203294
ZECUSDT,-0.799266,0.819338,0,35,"{'1%': -3.6327426647230316, '5%': -2.948510204...",4.040951
ZENUSDT,1.096363,0.995185,10,25,"{'1%': -3.7238633119999998, '5%': -2.98648896,...",-55.630497
ZILUSDT,0.468525,0.983884,7,28,"{'1%': -3.6889256286443146, '5%': -2.971989489...",-344.782229


In [6]:
fig = chartFigure(Close=dict(color='lightblue', width=3), height=400)

In [7]:
def on_changed(event, grid):
    """Redraw chart with selected ticker's data"""

    changed = grid.get_changed_df()
    k = event['new'][0]
    selected = changed.iloc[k:k + 1].to_dict('records')[0]
    ticker = selected['index']
    updateFigure(fig, Close=Close[ticker])
    fig.layout.title.text = ticker

In [9]:
X = ADF.reset_index()
grid = show_grid(X, grid_options={'editable': False, 'forceFitColumns': True, 'multiSelect': False, 'rowHeight': 24, 'maxVisibleRows': 14},
                 column_options={'defaultSortAsc': False},
                 column_definitions={'Critical Values': {'width': 500}})

grid.on('selection_changed', on_changed)
grid.change_selection([0])

In [10]:
VBox([fig, grid])

    'data': [{'line': {'color': 'lightblue', 'width': 3},
             …